In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import re
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
import pandas as pd
df = pd.read_csv("https://github.com/evelynh037/dsc106-final/raw/main/src/data/tweets.csv")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fejiang/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/fejiang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import re
stop_words = set(stopwords.words('english'))
import pandas as pd
df = pd.read_csv("https://github.com/evelynh037/dsc106-final/raw/main/src/data/tweets.csv")

In [37]:
import numpy as np

In [2]:
df.head()

,id,handle,text,is_retweet,original_author,time,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,...,place_type,place_country_code,place_country,place_contained_within,place_attributes,place_bounding_box,source_url,truncated,entities,extended_entities
0,780925634159796224,HillaryClinton,The question in this election: Who can put the...,False,NaN,2016-09-28T00:22:34,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://studio.twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/Xr...,{'media': [{'display_url': 'pic.twitter.com/Xr...
1,780916180899037184,HillaryClinton,"Last night, Donald Trump said not paying taxes...",True,timkaine,2016-09-27T23:45:00,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,http://twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/t0...,{'media': [{'display_url': 'pic.twitter.com/t0...
2,780911564857761793,HillaryClinton,Couldn't be more proud of @HillaryClinton. Her...,True,POTUS,2016-09-27T23:26:40,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://about.twitter.com/products/tweetdeck,False,"{'user_mentions': [{'id_str': '1536791610', 'n...",NaN
3,780907038650068994,HillaryClinton,"If we stand together, there's nothing we can't...",False,NaN,2016-09-27T23:08:41,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://studio.twitter.com,False,{'media': [{'display_url': 'pic.twitter.com/Q3...,{'media': [{'display_url': 'pic.twitter.com/Q3...
4,780897419462602752,HillaryClinton,Both candidates were asked about how they'd co...,False,NaN,2016-09-27T22:30:27,NaN,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,https://about.twitter.com/products/tweetdeck,False,"{'user_mentions': [], 'symbols': [], 'urls': [...",NaN


In [2]:
df[['handle', 'text']].to_csv("tweets.csv", index = False)

In [2]:
#get df for each other
hc = df[df["handle"] == "HillaryClinton"][["text","is_retweet","time"]]
dt = df[df["handle"] == "realDonaldTrump"][["text","is_retweet","time"]]
f'Hillary have {hc.shape[0]} tweets, Trump have {dt.shape[0]} tweets. {((hc["is_retweet"].sum()/hc.shape[0]) * 100).round(2)}% of Hillary tweets are retweets, {((dt["is_retweet"].sum()/dt.shape[0]) * 100).round(2)}% of Trump tweets are retweets'


'Hillary have 3226 tweets, Trump have 3218 tweets. 18.51% of Hillary tweets are retweets, 3.88% of Trump tweets are retweets'

In [3]:
#remove retweet
df = df[df["is_retweet"] == False]

In [4]:
#cleaned text
def clean_and_tokenize(tweet):
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    # Remove usernames
    tweet = re.sub(r'@\w+', '', tweet)
    # Remove hashtags
    tweet = re.sub(r'#\w+', '', tweet)
    # Remove punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)
    # Tokenize by splitting on whitespace
    tokens = tweet.split()
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    return tokens
hc["text_cleaned"] = hc["text"].apply(clean_and_tokenize)
dt["text_cleaned"] = dt["text"].apply(clean_and_tokenize)

In [5]:
hc_words = [token for word in hc["text_cleaned"] for token in set(word) if token not in stop_words]
hc_counts = Counter(hc_words)
hc_most_common_word = hc_counts.most_common(100000000000)
hc_summary = pd.DataFrame(hc_most_common_word, columns=['Word', 'Frequency'])
hc_summary["percent"] = hc_summary["Frequency"]/hc.shape[0]*100
hc_summary.to_csv('hillary_words.csv', index = False)
hc_summary

,Word,Frequency,percent
0,trump,703,21.791692
1,hillary,679,21.047737
2,donald,414,12.833230
3,president,263,8.152511
4,trumps,197,6.106634
...,...,...,...
5981,afroamericanas,1,0.030998
5982,afectadas,1,0.030998
5983,luchan,1,0.030998
5984,renta,1,0.030998


In [6]:
dt_words = [token for word in dt["text_cleaned"] for token in set(word) if token not in stop_words]
dt_counts = Counter(dt_words)
dt_most_common_word = dt_counts.most_common(10000000000)
dt_summary = pd.DataFrame(dt_most_common_word, columns=['Word', 'Frequency'])
dt_summary["percent"] = dt_summary["Frequency"]/dt.shape[0] * 100
dt_summary = dt_summary.sort_values("percent", ascending = False)
dt_summary.to_csv('trump_words.csv', index = False)
dt_summary

,Word,Frequency,percent
0,thank,517,16.065879
1,great,440,13.673089
2,hillary,326,10.130516
3,trump,326,10.130516
4,amp,240,7.458048
...,...,...,...
3430,presidentsand,1,0.031075
3431,slogan,1,0.031075
3432,americanism,1,0.031075
3433,globalism,1,0.031075


In [ ]:
dt_summary.to_json('data.json', orient='records', lines=True)

In [14]:
#exclamation mark
hc_exclamation = hc[hc["text"].str.contains("!", case=False, na=False)].shape[0]
dt_exclamation = dt[dt["text"].str.contains("!", case=False, na=False)].shape[0]
f'{hc_exclamation / hc.shape[0] * 100}% of Hillary tweets have ！mark, {dt_exclamation / dt.shape[0]* 100}% of Trump tweets have ！mark'

'4.153750774953503% of Hillary tweets have ！mark, 62.83405842137974% of Trump tweets have ！mark'

In [9]:
words_common=dt_summary.merge(hc_summary, left_on='Word', right_on='Word',suffixes=('_dt', '_hc'))
words_common

,Word,Frequency_dt,percent_dt,Frequency_hc,percent_hc
0,thank,514,16.618170,41,1.559528
1,great,434,14.031684,53,2.015976
2,hillary,320,10.345942,597,22.708254
3,trump,297,9.602328,583,22.175732
4,amp,231,7.468477,35,1.331305
...,...,...,...,...,...
2178,resources,1,0.032331,2,0.076075
2179,pledged,1,0.032331,2,0.076075
2180,kept,1,0.032331,1,0.038037
2181,disqualifying,1,0.032331,2,0.076075


In [26]:
words_common = pd.read_csv('common.csv')
words_common['category'] = words_common['category'].replace('economy', 'econ')

In [17]:
!pip install us

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.2 MB 24.0 MB/s eta 0:00:01
  Created wheel for us: filename=us-3.1.1-py3-none-any.whl size=12537 sha256=aa10cf2d08d93fc272592bbb61a7b613c672101f457e08c82103574cb174086f
  Stored in directory: /home/fejiang/.cache/pip/wheels/29/9e/92/3672525fc19ac574d668402d739c8e8ff4650012304d2f3f06
Successfully built us
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [27]:
import us
# Get a list of all state names
state_names = [state.name.lower() for state in us.states.STATES]

# Get a list of state abbreviations
state_abbre = [state.abbr.lower() for state in us.states.STATES]
select_state = lambda x: 'state' if x['Word'] in state_names or x['Word'] in state_abbre else x['category']
words_common['category'] = words_common.apply(select_state, axis = 1)

In [28]:
words_common['category'] = words_common['category'].replace('military', 'millitary')
words_common['category'] = words_common['category'].fillna('other')

In [29]:
words_common.category.value_counts()

category
other        1613
name           98
neg            69
endorse        51
promise        50
event          49
security       44
state          44
foreign        41
econ           37
policy         37
justice        29
poverty        25
job            24
health         21
patriotic      21
union          20
gender         18
media          17
race           15
edu            14
trade          13
millitary      12
slogan         11
time           11
immig          10
environ         9
tax             4
age             4
lgbt            4
attack          1
Name: count, dtype: int64

In [30]:
# into large categories:
demographic = ['gender', 'race', 'age', 'lgbt']
campaign_msg = ['slogan', 'policy', 'promise', 'event']
national_identity = ['patriotic', 'union', 'state']
economic_policy = ['econ', 'tax', 'job', 'trade']
security_policy = ['foreign', 'millitary', 'security', 'immig']
opponents_ref = ['name']
social_issue = ['justice', 'health', 'media', 'environ', 'poverty', 'edu']
emotional_word = ['endorse', 'neg', 'attack']

In [31]:
def add_larger_cate(x):
    if x == 'other' or x == 'time':
        return 'other_topic'
    elif x in demographic:
        return 'demographic_issue'
    elif x in campaign_msg:
        return 'campaign_message'
    elif x in national_identity:
        return 'national_identity'
    elif x in economic_policy:
        return 'economic_policy'
    elif x in security_policy:
        return 'security_policy'
    elif x in social_issue:
        return 'social_issue'
    elif x in emotional_word:
        return 'emotional_word'
    else:
        return 'opponents_reference'

words_common['topic'] = words_common['category'].apply(add_larger_cate)

In [32]:
# One hot encoding sub-categories and topics
from sklearn.preprocessing import OneHotEncoder

myohe = OneHotEncoder()
myoheclass = myohe.fit_transform(words_common.category.values.reshape(-1,1)).toarray()
df_encoded = pd.concat([words_common, pd.DataFrame(myoheclass, columns=list(myohe.categories_[0]))], axis=1)

myohe2 = OneHotEncoder()
myohetopic = myohe2.fit_transform(words_common.topic.values.reshape(-1,1)).toarray()
all_common = pd.concat([df_encoded, pd.DataFrame(myohetopic, columns=list(myohe2.categories_[0]))], axis = 1)


In [34]:
all_common = all_common.drop(columns = ['names'])

In [38]:
all_common['total_freq'] = all_common['Frequency_dt'] + all_common['Frequency_hc']
all_common['diff_freq'] = np.abs(all_common['Frequency_dt'] - all_common['Frequency_hc'])

In [81]:
all_common

,Word,Frequency_dt,percent_dt,Frequency_hc,percent_hc,category,topic,age,attack,econ,...,demographic_issue,economic_policy,emotional_word,national_identity,opponents_reference,other_topic,security_policy,social_issue,total_freq,diff_freq
0,thank,517,16.065879,47,1.456913,event,campaign_message,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,564,470
1,great,440,13.673089,63,1.952883,promise,campaign_message,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,503,377
2,hillary,326,10.130516,679,21.047737,name,opponents_reference,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1005,353
3,trump,326,10.130516,703,21.791692,name,opponents_reference,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1029,377
4,amp,240,7.458048,77,2.386857,other,other_topic,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,317,163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,kept,1,0.031075,1,0.030998,other,other_topic,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,0
2412,threatens,1,0.031075,1,0.030998,security,security_policy,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2,0
2413,disqualifying,1,0.031075,2,0.061996,neg,emotional_word,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3,1
2414,conduct,1,0.031075,5,0.154991,other,other_topic,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6,4


In [82]:
all_common.to_csv('words_in_common.csv', index=False)

---
Above is all stats on Words used in both candidates' tweets
Now move on to slogans in their tweets

In [71]:
# from this: https://en.wikipedia.org/wiki/List_of_U.S._presidential_campaign_slogans#:~:text=%22Ready%20for%20change%2C%20ready%20to,experience%20to%20make%20change%20happen.%22
hc_slogan = ['hillary for america', 'stronger together', 'forward together', "i'm with her", 'fighting for us', 'stick it to the man by voting for a woman', 'love trumps hate', 'when they go low, we go high']
dt_slogan = ['make america great again', 'maga', 'america first', 'promises made, promises kept']
hc_used = []
dt_used = []
def check_hc_slogan(tweet):
    match = [slogan for slogan in hc_slogan if slogan in tweet.lower()]
    if len(match)!=0:
        for slog in match:
            if slog not in hc_used:
                hc_used.append(slog)
    return match if len(match)!= 0 else 0

def check_dt_slogan(tweet):
    match = [slogan for slogan in dt_slogan if slogan in tweet.lower()]
    if len(match)!=0:
        for slog in match:
            if slog not in dt_used:
                dt_used.append(slog)
    return match if len(match)!= 0 else 0

# Apply the function to create a new column 'included_slogans'
hc['slogans'] = hc['text'].apply(check_hc_slogan)
dt['slogans'] = dt['text'].apply(check_dt_slogan)

In [72]:
hc_slogan_sum = hc[hc['slogans']!=0]
dt_slogan_sum = dt[dt['slogans']!=0]

In [73]:
# one-hot-encode slogan
for slog in hc_used:
    hc_slogan_sum[slog] = hc_slogan_sum['slogans'].apply(lambda slogan: int(slog in slogan))

for slog in dt_used:
    dt_slogan_sum[slog] = dt_slogan_sum['slogans'].apply(lambda slogan: int(slog in slogan))

/tmp/ipykernel_212/1894537887.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hc_slogan_sum[slog] = hc_slogan_sum['slogans'].apply(lambda slogan: int(slog in slogan))
/tmp/ipykernel_212/1894537887.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hc_slogan_sum[slog] = hc_slogan_sum['slogans'].apply(lambda slogan: int(slog in slogan))
/tmp/ipykernel_212/1894537887.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [101]:
print("Trump's slogan stats:")
print(f"Total {dt_slogan_sum.shape[0]} posts contains some type of slogan")
print(f"MAGA: {dt_slogan_sum.maga.sum()}; Make America Great Again: {dt_slogan_sum['make america great again'].sum()};")
print(f"America First: {dt_slogan_sum['america first'].sum()}")

Trump's slogan stats:
Total 168 posts contains some type of slogan
MAGA: 66; Make America Great Again: 97;
America First: 7


In [108]:
print("Clinton's slogan stats:")
print(f"Total {hc_slogan_sum.shape[0]} posts contains some type of slogan")
print(f"love trumps hate: {hc_slogan_sum['love trumps hate'].sum()}; Stronger Together: {hc_slogan_sum['stronger together'].sum()}")
print(f"Hillary for America: {hc_slogan_sum['hillary for america'].sum()}; I'm with her: ", hc_slogan_sum["i'm with her"].sum())

Clinton's slogan stats:
Total 39 posts contains some type of slogan
love trumps hate: 9; Stronger Together: 28
Hillary for America: 1; I'm with her:  1


In [192]:
slogan_stats = pd.DataFrame({"slogan": ['MAGA', 'Make America Great Again', 'America First', 'Love Trumps Hate', 'Stronger Together', 'Hillary for America', "I'm with her"],
                            "num_tweets": [66, 97, 7, 9, 28, 1, 1], 'Clinton':[0,0,0,1,1,1,1], "Trump":[1,1,1,0,0,0,0]})
slogan_stats

,slogan,num_tweets,Clinton,Trump
0,MAGA,66,0,1
1,Make America Great Again,97,0,1
2,America First,7,0,1
3,Love Trumps Hate,9,1,0
4,Stronger Together,28,1,0
5,Hillary for America,1,1,0
6,I'm with her,1,1,0


In [193]:
slogan_stats.to_csv("slogan_stats.csv", index = False)

In [79]:
all_slogans = pd.concat([hc_slogan_sum, dt_slogan_sum]).fillna(0)
all_slogans['Cliton'] = np.array([1]*hc_slogan_sum.shape[0] + [0]*dt_slogan_sum.shape[0])
all_slogans['Trump'] = np.array([0]*hc_slogan_sum.shape[0] + [1]*dt_slogan_sum.shape[0])
all_slogans

,text,is_retweet,time,text_cleaned,slogans,love trumps hate,stronger together,hillary for america,i'm with her,maga,make america great again,america first,Cliton,Trump
265,We believe that everyone has value.\nWe believ...,False,2016-09-21T19:12:46,"[we, believe, that, everyone, has, value, we, ...",[love trumps hate],1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
326,Let’s make clear that love trumps hate—not jus...,False,2016-09-19T19:03:31,"[lets, make, clear, that, love, trumps, hateno...",[love trumps hate],1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
424,"""We deserve a president…who believes that each...",False,2016-09-16T19:27:44,"[we, deserve, a, presidentwho, believes, that,...",[stronger together],0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,0
450,Let’s build a future where love trumps hate. h...,False,2016-09-16T14:05:45,"[lets, build, a, future, where, love, trumps, ...",[love trumps hate],1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
607,“She knows that love trumps hate.” —@POTUS on ...,False,2016-09-13T18:12:17,"[she, knows, that, love, trumps, hate, on, hil...",[love trumps hate],1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6400,"""@hofmannken: DonaldTrump, wish I could attend...",False,2016-01-08T03:12:06,"[donaldtrump, wish, i, could, attend, one, of,...",[make america great again],0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1
6402,We could only get a small fraction of this 25k...,False,2016-01-08T02:23:07,"[we, could, only, get, a, small, fraction, of,...",[make america great again],0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1
6413,"I will be in beautiful Burlington, Vermont, to...",False,2016-01-07T13:17:40,"[i, will, be, in, beautiful, burlington, vermo...",[make america great again],0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1
6440,"""@marybnall01: @realDonaldTrump watched lowell...",False,2016-01-05T03:44:17,"[watched, lowell, mass, speech, awesome, great...",[make america great again],0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,1


In [80]:
all_slogans.to_csv('all_slogans.csv', index=False)

---
now move forward and do words that are unique to each candidate

In [84]:
hc_unique = hc_summary[~hc_summary['Word'].isin(dt_summary['Word'])]
dt_unique = dt_summary[~dt_summary['Word'].isin(hc_summary['Word'])]

In [165]:
# we take only more common words with freq>3
hc_words = pd.read_csv('clinton_unique.csv')
dt_words = pd.read_csv('trump_unique.csv')

In [166]:
def add_larger_cate(x):
    if x == 'other' or x == 'time':
        return 'other_topic'
    elif x in demographic or x=='religion':
        return 'demographic_issue'
    elif x in campaign_msg or x=='media' or x=='spanish':
        return 'campaign_message'
    elif x in national_identity:
        return 'national_identity'
    elif x in economic_policy:
        return 'economic_policy'
    elif x in security_policy:
        return 'security_policy'
    elif x in social_issue:
        return 'social_issue'
    elif x in emotional_word:
        return 'emotional_word'
    else:
        return 'opponents_reference'

hc_words['topic'] = hc_words['category'].apply(add_larger_cate)
dt_words['topic'] = dt_words['category'].apply(add_larger_cate)

In [167]:
myohe = OneHotEncoder()
myoheclass = myohe.fit_transform(hc_words.category.values.reshape(-1,1)).toarray()
hc_words = pd.concat([hc_words, pd.DataFrame(myoheclass, columns=list(myohe.categories_[0]))], axis=1)

myohe2 = OneHotEncoder()
myohetopic = myohe2.fit_transform(hc_words.topic.values.reshape(-1,1)).toarray()
hc_words = pd.concat([hc_words, pd.DataFrame(myohetopic, columns=list(myohe2.categories_[0]))], axis = 1)

myohe = OneHotEncoder()
myoheclass = myohe.fit_transform(dt_words.category.values.reshape(-1,1)).toarray()
dt_words = pd.concat([dt_words, pd.DataFrame(myoheclass, columns=list(myohe.categories_[0]))], axis=1)

myohe2 = OneHotEncoder()
myohetopic = myohe2.fit_transform(dt_words.topic.values.reshape(-1,1)).toarray()
dt_words = pd.concat([dt_words, pd.DataFrame(myohetopic, columns=list(myohe2.categories_[0]))], axis = 1)

In [168]:
# put together with the words in common, all_dt has all the words trump posted
temp = all_common.rename(columns={"Frequency_dt": "Frequency", "percent_dt": "percent"})
cols = dt_words.columns.to_list()
cols.remove('religion')
temp = temp[cols]
all_dt = pd.concat([temp, dt_words]).fillna(0)

In [169]:
temp = all_common.rename(columns={"Frequency_hc": "Frequency", "percent_hc": "percent"})
cols = hc_words.columns.to_list()
cols.remove('spanish')
cols.remove('religion')
temp = temp[cols]
all_hc = pd.concat([temp, hc_words]).fillna(0)

In [187]:
count = lambda x: x.count()
trump_cate = all_dt.groupby('category').agg({"Frequency":sum, "percent":count, "topic":min}).rename(columns={"percent":"count"})
clinton_cate = all_hc.groupby('category').agg({"Frequency":sum, "percent":count, "topic":min}).rename(columns={"percent":"count"})
category_summary = pd.concat([clinton_cate, trump_cate], axis=1).fillna(0)

In [188]:
category_summary.columns = ['clinton_freq', 'clinton_count', 'topic', 'trump_freq', 'trump_count', 'topic2']
category_summary = category_summary[category_summary.columns[:-1]]
category_summary['trump_freq'] = category_summary['trump_freq'].astype(int)
category_summary['trump_count'] = category_summary['trump_count'].astype(int)

myohe = OneHotEncoder()
myoheclass = myohe.fit_transform(category_summary.topic.values.reshape(-1,1)).toarray().astype(int)
cate_summary = pd.concat([category_summary.reset_index(), pd.DataFrame(myoheclass, columns=list(myohe.categories_[0]))], axis=1)
cate_summary

,category,clinton_freq,clinton_count,topic,trump_freq,trump_count,campaign_message,demographic_issue,economic_policy,emotional_word,national_identity,opponents_reference,other_topic,security_policy,social_issue
0,age,51,6,demographic_issue,14,4,0,1,0,0,0,0,0,0,0
1,attack,105,16,emotional_word,534,27,0,0,0,1,0,0,0,0,0
2,econ,429,49,economic_policy,223,40,0,0,1,0,0,0,0,0,0
3,edu,305,21,social_issue,53,15,0,0,0,0,0,0,0,0,1
4,endorse,903,52,emotional_word,1064,60,0,0,0,1,0,0,0,0,0
5,environ,57,12,social_issue,50,12,0,0,0,0,0,0,0,0,1
6,event,1057,51,campaign_message,2385,60,1,0,0,0,0,0,0,0,0
7,foreign,208,44,security_policy,350,51,0,0,0,0,0,0,0,1,0
8,gender,394,29,demographic_issue,174,20,0,1,0,0,0,0,0,0,0
9,health,255,23,social_issue,93,23,0,0,0,0,0,0,0,0,1


In [189]:
cate_summary.to_csv("category_sum_all.csv", index = False)

In [190]:
count = lambda x: x.count()
trump_topic = all_dt.groupby('topic').agg({"Frequency":sum, "percent":count}).rename(columns={"percent":"count"})
clinton_topic = all_hc.groupby('topic').agg({"Frequency":sum, "percent":count}).rename(columns={"percent":"count"})
topic_summary = pd.concat([clinton_topic, trump_topic], axis=1)
topic_summary.columns = ['clinton_freq', 'clinton_count', 'trump_freq', 'trump_count']
topic_summary = topic_summary.reset_index()
topic_summary

,topic,clinton_freq,clinton_count,trump_freq,trump_count
0,campaign_message,4362,208,5552,180
1,demographic_issue,727,65,314,52
2,economic_policy,1145,102,590,85
3,emotional_word,1475,137,2418,156
4,national_identity,1684,97,1737,111
5,opponents_reference,3458,108,2889,130
6,other_topic,10515,1743,9577,1743
7,security_policy,824,114,845,122
8,social_issue,1605,140,949,128


In [191]:
topic_summary.to_csv('topic_sum_all.csv', index = False)

In [15]:
#quote of their own
hc_quote = hc[hc["text"].str.contains("—hillary", case=False, na=False)].shape[0]
dt_quote = dt[dt["text"].str.contains("-trump", case=False, na=False)].shape[0]
f'{hc_quote / hc.shape[0] * 100}% of Hillary tweets is quoting of her own words, {dt_quote / dt.shape[0]* 100}% of Trump tweets is quoting of his words'


'9.702417854928704% of Hillary tweets is quoting of her own words, 0.06215040397762585% of Trump tweets is quoting of his words'

In [16]:
#quote of each other
hc_qo = hc[hc["text"].str.contains("—Trump", case=False, na=False)].shape[0]
dt_qo = dt[dt["text"].str.contains("-Clinton", case=False, na=False)].shape[0]
f'{hc_qo / hc.shape[0] * 100}% of Hillary tweets is quoting of Trump, {dt_qo / dt.shape[0]* 100}% of Trump tweets is quoting of Hillary'

'0.24798512089274644% of Hillary tweets is quoting of Trump, 0.031075201988812924% of Trump tweets is quoting of Hillary'

In [20]:
##campaign slogan
hc_slogan = hc[hc["text"].str.contains("Stronger Together", case=False, na=False)].shape[0]
dt_slogan = dt[dt["text"].str.lower().str.contains("great again",case=False, na=False)].shape[0]
f'{hc_slogan/ hc.shape[0] * 100}% of Hillary tweets mentioned Stronger Together, {dt_slogan / dt.shape[0]* 100}% of Trump tweets meantioned Great Again'


'0.8679479231246126% of Hillary tweets mentioned Stronger Together, 3.6047234307022995% of Trump tweets meantioned Great Again'

In [24]:
#trump immigration promise
immig_words = ["border", "deport", "mexico", "wall", "refugee","illegal", 'immigrat']
dt_immig = dt_summary[dt_summary['Word'].apply(lambda tweet: any(word in tweet.lower() for word in immig_words))]
dt_immig_words = [i for i in dt_immig["Word"]]
dt_immig_p = dt_immig.sum()
hc_immig = hc_summary[hc_summary['Word'].apply(lambda tweet: any(word in tweet.lower() for word in immig_words))]
hc_immig_words = [i for i in hc_immig["Word"]]
hc_immig_p = hc_immig.sum()
f'About immigration, Trump often use {dt_immig_words} and it consists of {dt_immig_p.loc["percent"]*100} of his tweets,Hillary often use {hc_immig_words} and it consists of {hc_immig_p.loc["percent"]*100} of her tweets, one of it is about Lets imagine a tomorrow in which no child grows up under the shadows of discrimination or deportation'


"About immigration, Trump often use ['wall', 'illegal', 'immigration', 'mexico', 'borders', 'border', 'refugees', 'illegals', 'wallace', 'illegally', 'refugee', 'walls', 'wallsis', 'wallet', 'interviewall', 'borderless', 'immigrationif', 'prowall', 'deportation'] and it consists of 546.9235550031075 of his tweets,Hillary often use ['wall', 'immigration', 'walls', 'deport', 'mexico', 'deportation', 'deported', 'illegal', 'deportar', 'stonewall', 'deportación', 'refugees', 'immigrate', 'wallet', 'wallinstead', 'walldid', 'deporting', 'refugee', 'border', 'deportations'] and it consists of 325.4804711717297 of her tweets, one of it is about Lets imagine a tomorrow in which no child grows up under the shadows of discrimination or deportation"

In [22]:
#trump tax cut
tax_words = ["tax"]
dt_tax = dt_summary[dt_summary['Word'].apply(lambda tweet: any(word in tweet.lower() for word in tax_words))]
dt_tax_words = [i for i in dt_tax["Word"]]
dt_tax_p = dt_tax.sum()
hc_tax = hc_summary[hc_summary['Word'].apply(lambda tweet: any(word in tweet.lower() for word in tax_words))]
hc_tax_words = [i for i in hc_tax["Word"]]
hc_tax_p = hc_tax.sum()
f'About taxation, Trump often use {dt_tax_words} and it consists of {dt_immig_p.loc["percent"]*100} of his tweets,Hillary often use {hc_tax_words} and it consists of {hc_tax_p.loc["percent"]*100} of her tweets, one of it is about Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic.'



'About immigration, Trump often use [\'tax\', \'taxes\', \'overtaxes\', \'overtaxed\', \'taxpayers\'] and it consists of 463.02050963331254 of his tweets,Hillary often use [\'tax\', \'taxes\', \'taxpayers\', \'taxpayer\', \'taxesand\'] and it consists of 285.18288902665836 of her tweets, one of it is about Last night, Donald Trump said not paying taxes was "smart." You know what I call it? Unpatriotic.'

In [50]:
# hillary health promise(need to be modified)
health_words = ["heath","affordable","care", 'insurance','medical']
dt_health = dt_summary[dt_summary['Word'].apply(lambda tweet: any(word in tweet.lower() for word in health_words))]
dt_health_words = [i for i in dt_health["Word"]]
dt_health_p = dt_health.sum()
hc_health = hc_summary[hc_summary['Word'].apply(lambda tweet: any(word in tweet.lower() for word in health_words))]
hc_health_words = [i for i in hc_health["Word"]]
hc_health_p = hc_health.sum()
f'About health plan, Trump often use {dt_health_words} and it consists of {dt_immig_p.loc["percent"]*100}% of his tweets,Hillary often use {hc_health_words} and it consists of {hc_health_p.loc["percent"]*100} of her tweets.'



"About health plan, Trump often use ['care', 'obamacare', 'careful', 'healthcare', 'cares', 'career', 'scared', 'insurance', 'pledgecareful', 'ocare', 'childcare', 'medical', 'careers', 'careless', 'scare'] and it consists of 546.9235550031075% of his tweets,Hillary often use ['care', 'affordable', 'career', 'insurance', 'cares', 'medical', 'scared', 'careerfocused', 'scares', 'careerand', 'childcare', 'medicare', 'heath', 'scare'] and it consists of 322.38065716057037 of her tweets."